In [9]:
import pandas as pd
import requests
import pymongo
from sqlalchemy import create_engine



url = 'https://data.cityofchicago.org/api/views/5udb-dr6f/rows.json?accessType=DOWNLOAD'
r= requests.get(url).json()


In [11]:
df = pd.DataFrame(r['data'])
df = df.drop(df.columns[[0, 1, 2, 3, 5, 7]], axis=1, inplace=False)
df =df.dropna(axis=1, how='all', thresh=None, subset=None, inplace=False)
df.rename(columns={8:'Name',9:'DBA',10:'identifier', 11:'Type',12:'Risk', 13:'Street',14:'City', 15:'State', 16:'Zip', 17:'Date',18:'Inspection',19:'Success',20:'Description', 21:'lat', 22:'lng'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date']).apply(lambda x:x.strftime('%m/%d/%Y'))
df['Year'] = pd.to_datetime(df['Date']).apply(lambda x:x.strftime('%Y'))
df.head(5)

,Name,DBA,identifier,Type,Risk,Street,City,State,Zip,Date,Inspection,Success,Description,lat,lng,Year
0,"WALLACE CATFISH, INC.","WALLACE CATFISH, INC.",1954531,Restaurant,Risk 1 (High),2800 W MADISON ST,CHICAGO,IL,60612,01/08/2013,Complaint Re-Inspection,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.881185442760426,-87.69636210906256,2013
1,I-57 RIB HOUSE #5,I-57 RIB HOUSE #5,1696182,Restaurant,Risk 1 (High),6514 S WESTERN AVE,CHICAGO,IL,60636,01/08/2013,Complaint,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.77511152013113,-87.68367904074279,2013
2,KALIFLOWER,KALIFLOWER,2670294,Restaurant,Risk 1 (High),333 N MICHIGAN AVE,CHICAGO,IL,60601,06/17/2019,License,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.88754294139151,-87.62440700245574,2019
3,JUST TURKEY RESTAURANT,JUST TURKEY RESTAURANT,2054263,Restaurant,Risk 1 (High),9615 S WESTERN AVE,CHICAGO,IL,60643,01/07/2013,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.71862594545408,-87.68184548069195,2013
4,YELLOW ROSE CAFE INC,YELLOW ROSE CAFE,50484,Restaurant,Risk 1 (High),5640 N ELSTON AVE,CHICAGO,IL,60646,01/07/2013,Canvass,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.98376846724376,-87.7661756872335,2013


In [37]:

df = df[df.Year != '2011']
df = df[df.Year != '2012']
df = df[df.Year != '2013']
df = df[df.Year != '2014']
df = df[df.Year != '2015']



In [40]:
df2.describe()

,Name,DBA,identifier,Type,Risk,Street,City,State,Zip,Date,Inspection,Success,Description,lat,lng,Year
count,48924,48709,48924,48924,48924,48924,48885,48903,48914,48924,48923,48924,37090,48861,48861,48924
unique,9483,9088,12513,1,4,8288,8,1,59,854,16,7,36916,8142,8142,4
top,SUBWAY,SUBWAY,0,Restaurant,Risk 1 (High),11601 W TOUHY AVE,CHICAGO,IL,60614,09/13/2016,Canvass,Pass,45. FOOD HANDLER REQUIREMENTS MET - Comments: ...,42.008536400868735,-87.91442843927047,2016
freq,1017,1257,22,48924,40564,960,48793,48903,2364,113,26573,22957,7,960,960,16174


In [112]:
###########Method 1 MONGO DB   #################

#insert into Mongo
#REMEMBER TO START MONGOD in your bash!!!!!!!!
conn = 'mongodb://localhost:27017'
client=pymongo.MongoClient(conn)
#define DB and collection/table to work with
db = client.ChicagoRest
#convert DataFrame to JSON format for insert
#Load all dataframes
records = json.loads(df.to_json(orient='records'))
db.CompanyData.insert_many(records)

In [3]:
#sqlite
connection_string = "sqlite:///ChicagoRestaurants.sqlite"
engine = create_engine(connection_string)
#list tables names- if you want
#engine.table_names()

####Create the database to match our columns in our dataframe####
#Name	DBA	ID	Type	Risk	Street	City	State	Zip	Date	Inspection	Success	Description	lat	lng


##############IMPORTANT###############################
########### hash out this after creating as it will give an error stating the DB already exists after running the first time ####
####apparently the line below is not needed to create the sql lite.. if unhash it creates another table in the db
engine.execute("CREATE TABLE RestaurantData(id INT PRIMARY KEY, Name TEXT, DBA TEXT, identifier TEXT, Type TEXT, Risk TEXT, Street TEXT, City TEXT, State TEXT, Zip TEXT, Date TEXT, Inspection TEXT, Success TEXT, Description TEXT, Lat Int, Lng Int);")
#engine.execute("CREATE TABLE RestaurantData(Name TEXT, DBA TEXT, identifier TEXT, Type TEXT, Risk TEXT, Street TEXT, City TEXT, State TEXT, Zip TEXT, Date TEXT, Inspection TEXT, Success TEXT, Description TEXT, Lat Int, Lng Int);")

################################################################################################################

####### PLUG that data into SQLITE#####
df.to_sql(name="RestaurantData", con=engine, if_exists="append", index=False)
#####SHOW ME THE MONEY#####
pd.read_sql_query("SELECT * FROM RestaurantData",con=engine).head()

,id,Name,DBA,identifier,Type,Risk,Street,City,State,Zip,Date,Inspection,Success,Description,Lat,Lng
0,None,FULTON GALLEY,None,2636278,Restaurant,Risk 3 (Low),1115 W FULTON MARKET,CHICAGO,IL,60607,06/07/2019,License Re-Inspection,Pass w/ Conditions,None,41.886571,-87.655174
1,None,WALDORF ASTORIA CHICAGO,MARGEAUX/ CAFE/EMPLOYEE CAFE,2134586,Restaurant,Risk 1 (High),11 E WALTON ST,CHICAGO,IL,60611,02/14/2012,License,Pass,None,41.899835,-87.627799
2,None,LAS FRESAS,LAS FRESAS,2247279,Restaurant,Risk 2 (Medium),7057 N CLARK ST,CHICAGO,IL,60626,06/06/2019,Canvass,Out of Business,None,42.010675,-87.674167
3,None,GORILLA SUSHI LOGAN SQUARE,GORILLA SUSHI LOGAN SQUARE,2543141,Restaurant,Risk 1 (High),2735 N MILWAUKEE AVE,CHICAGO,IL,60647,06/06/2019,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.930969,-87.710286
4,None,PICNIC WINE & PROVISIONS,PICNIC WINE & PROVISIONS,2636701,Restaurant,Risk 2 (Medium),7301 N SHERIDAN RD,CHICAGO,IL,60626,06/06/2019,License,Fail,None,42.014041,-87.663971


In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [6]:
# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///ChicagoRestaurants.sqlite")

In [7]:
########################################################################################################
######################QUERY 1. THIS PULLS ALL DATA AND RETURNS JSONIFIED################################
########################################################################################################
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
restaurants = Base.classes.RestaurantData
session = Session(engine)
###Query####
Output = []
#Name	DBA	identifier	Type	Risk	Street	City	State	Zip	Date	Inspection	Success	Description	Lat	Lng
for row in session.query(restaurants, restaurants.Name, restaurants.DBA, restaurants.Type,restaurants.Risk,restaurants.Street, restaurants.City, \
                         restaurants.State, restaurants.Zip, restaurants.Date,restaurants.Inspection, restaurants.Success, restaurants.Description, \
                        restaurants.Lat, restaurants.Lng).all():
    Output.append(row)
df = pd.DataFrame(data=Output)
df.to_json(orient='records')

In [ ]:
########################################################################################################
######################QUERY 2. THIS PULLS ALL DATA BY YEAR################################
########################################################################################################